In [ ]:
import json
import csv
import pandas as pd
import pyodbc
import mysql.connector as msql

In [ ]:
# Ingest diabetes.csv
# Dataset from : https://www.kaggle.com/datasets/akshaydattatraykhare/diabetes-dataset
data = pd.read_csv("diabetes.csv")
data

In [ ]:
# Transform data. 

# This sections adds an identifier column to the dataset
identifier_values = []
for i in range (1,769):
    identifier_values.append(i)
data.insert(0, "Identifier Number", identifier_values, True)

# This section changes the values in 'outcome' to be more
# clear so that the user can easily understand what they mean.
data = data.rename(columns={'Outcome': 'Diabetes Prediction'})
data['Diabetes Prediction'] = data['Diabetes Prediction'].replace(1, 'Yes')
data['Diabetes Prediction'] = data['Diabetes Prediction'].replace(0, 'No')

# Additional Column Transformation: Adding a column that indicates whether BMI and blood pressure are above normal.
# This could be useful in seeing whether there's a particular combo of predictors that lead to higher likelihood
# of diabetes.
higher_averages = []
count = 0
for rows in data['BMI']:
    if int(rows) >= 25 and int(data['Pregnancies'].iloc[count]) > 2:
        higher_averages.append('both above average')
    elif int(rows)>=25:
        higher_averages.append('Above average BMI')
    elif int(data['Pregnancies'].iloc[count]) > 2:
        higher_averages.append("Above average number of pregnancies")
    else:
        higher_averages.append("Neither above average")
    count = count+1
data.insert(10, "BMI/Pregnancies in comparison to average", higher_averages, True)
data

In [ ]:
# Ask user what they want their output to look like
print("What format do you want your output file? (Options: CSV, JSON, SQL database table)")
target = input()

if target == 'CSV':
    data.to_csv('transformed_diabetes.csv')
    print(data)
elif target == 'SQL database table':
    # Source for csv to SQL conversion: https://www.projectpro.io/recipes/connect-mysql-python-and-import-csv-file-into-mysql-and-create-table
    from mysql.connector import Error
    #Gets login info from user so sql file can be created using MySQL
    print("Please enter the host, user, and password you would like to use and separate each with spaces (i.e. localhost user123 password123): ")
    ans = input()
    # Deals with invalid user input
    if len(ans.split()) != 3:
        raise Exception('Incorrect number of arguments. Please run again and enter the host,user,and password separated by spaces.')
    host,user,password = ans.split()
    try:
        # Connect to user's SQL workbench and creates the diabetes database (not populated yet though)
        conn = msql.connect(host=host, user=user,  
                        password=password)#give ur username, password
        if conn.is_connected():
            cursor = conn.cursor()
            cursor.execute("CREATE DATABASE diabetes")
            print("Database is created")
    # Error in case connection fails
    except Error as e:
        print("Error while connecting to MySQL", e)
    try:
        # Connect to SQL workbench and populate diabetes database 
        conn = msql.connect(host=host, database='diabetes', user=user, password=password)
        if conn.is_connected():
            cursor = conn.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)
            #if the database exists prior to it being created in this code->drop it
            cursor.execute('DROP TABLE IF EXISTS diabetes_data;')
            print('Creating table....')
            # Names the columns
            cursor.execute("CREATE TABLE diabetes_data(Identifier_Name varchar(255),Pregnancies varchar(255),Glucose varchar(255),Blood_Pressure varchar(255),Skin_Thickness varchar(255),Insulin varchar(255),BMI varchar(255),DiabetesPedigreeFunction varchar(255),Age varchar(255),Diabetes_Prediction varchar(255),BMI_and_Pregnancies_in_comparison_to_average varchar(255))")
            print("Table is created....")
            #loop through the data frame and add values row by row
            for i,row in data.iterrows():
                #here %S means string values 
                # variable sql has the command we want to execute, inserting values into our table
                sql = "INSERT INTO diabetes.diabetes_data VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                cursor.execute(sql, tuple(row))
                print("Record inserted")
                # the connection is not auto committed by default, so we must commit to save our changes
                conn.commit()
    except Error as e:
            print("Error while connecting to MySQL", e)
# Converting to json
elif target == 'JSON':
    data.to_json('diabetes.json')
    # In order to make file more readable we turn json file back to dictionary and use .dump to be able to indent and write
    # that back to the file.
    with open("diabetes.json", "r") as read_file:
         json_load = json.load(read_file)
    with open("diabetes.json", "w") as outfile:
         json.dump(json_load,outfile, indent=4)
    pretty_json = json.dumps(json_load,indent=4)
    print(pretty_json)
    print('JSON file sucessfully created')
    
else:
    # Exception if user entered an invalid output file format
    raise Exception('Please run again and select a valid output file format (CSV, JSON, SQL database table)')


In [ ]:
# Generate a brief summary of data. Here we will output number of records and columns.
print("General Summary:")
print("This data intends to show whether or not the patient is predicted to have diabetes based on pregnancies, glucose, BMI, blood pressure, and age. It is important to note that this dataset represents women of Pima Indian heritage and that are above the age of 21.")
print("This data set has",len(data.index), 'records and', len(data.columns), "columns.")